In [1]:
## new attempt by converting CFHT data to xarray
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
 
import netCDF4
import xarray as xr

import xarray.plot as xplt
import sunpy.timeseries as ts 

########## for cycle ##############
from matplotlib import dates as d
import datetime as dt
import time

from functools import reduce


In [2]:
#open NETCDF files (all .nc files from path) for "specific humidity"
#ds_RH_600 = xr.open_mfdataset('/home/caroline/hulk/Astroclimate/cds_data_ERA5/RH/RH_600hPa/*.nc', combine = 'by_coords', parallel=True)
ds_RH_600 = xr.open_dataset('/home/caroline/hulk/Astroclimate/cds_data_ERA5/RH/Era5_2008to2012_RH_600hPa.nc')
ds_RH_650 = xr.open_dataset('/home/caroline/hulk/Astroclimate/cds_data_ERA5/RH/Era5_2008to2012_RH_650hPa.nc')
ds_RH_700 = xr.open_dataset('/home/caroline/hulk/Astroclimate/cds_data_ERA5/RH/Era5_2008to2012_RH_700hPa.nc')
ds_RH_750 = xr.open_dataset('/home/caroline/hulk/Astroclimate/cds_data_ERA5/RH/Era5_2008to2012_RH_750hPa.nc')

#observational data from MaunaKea (CFHT)
CFHT_hourly = pd.read_csv('/home/caroline/Dropbox/Astroclimate Project/Mauna_Kea/CFHT/pressure_levels_600to750hPa/downsampled_masked_RH_1991to2018_hourly_means.csv')

In [3]:
df_RH_600 = ds_RH_600.sel(longitude=204.53,latitude= 19.83,method='nearest')#.sel(time=slice('2008-01-01', '2012-12-31'))
df_RH_650 = ds_RH_650.sel(longitude=204.53,latitude= 19.83,method='nearest')
df_RH_700 = ds_RH_700.sel(longitude=204.53,latitude= 19.83,method='nearest')
df_RH_750 = ds_RH_750.sel(longitude=204.53,latitude= 19.83,method='nearest')

In [4]:
df_RH_700 # why behave df_RH_700 so differently than df_RH_600? because of open_mfdataset?

<xarray.Dataset>
Dimensions:    (time: 43848)
Coordinates:
    longitude  float32 204.5
    latitude   float32 19.75
  * time       (time) datetime64[ns] 2008-01-01 ... 2012-12-31T23:00:00
Data variables:
    r          (time) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-02-18 18:51:05 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [6]:
df_RH_600

<xarray.Dataset>
Dimensions:    (time: 43848)
Coordinates:
    longitude  float32 204.5
    latitude   float32 19.75
  * time       (time) datetime64[ns] 2008-01-01 ... 2012-12-31T23:00:00
Data variables:
    r          (time) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-02-25 16:09:40 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [7]:
# rename columns for better overview
df_RH_600 = df_RH_600.rename({'r': 'r_600'})
df_RH_650 = df_RH_650.rename({'r': 'r_650'})
df_RH_700 = df_RH_700.rename({'r': 'r_700'})
df_RH_750 = df_RH_750.rename({'r': 'r_750'})

In [8]:
# prepare CFHT observational data for merging

CFHT_hourly = CFHT_hourly.rename(columns={'Unnamed: 0': 'time'})
# change the format of the times column to datetime format
CFHT_hourly['time'] = pd.to_datetime(CFHT_hourly['time']) 

#check the format
print(CFHT_hourly['time'].dtype)
print(CFHT_hourly['time'][0])

#set index 
CFHT_hourly.set_index('time', inplace=True)

datetime64[ns]
1992-03-13 21:00:00


In [9]:
CFHT_xr = CFHT_hourly.to_xarray()

In [10]:
CFHT_xr

<xarray.Dataset>
Dimensions:               (time: 264789)
Coordinates:
  * time                  (time) datetime64[ns] 1992-03-13T21:00:00 ... 2020-01-17T02:00:00
Data variables:
    relative_humidity(%)  (time) float64 15.6 13.94 14.85 ... 98.43 98.35 94.54

In [10]:
df_RH_650

<xarray.Dataset>
Dimensions:    (time: 43848)
Coordinates:
    longitude  float32 204.5
    latitude   float32 19.75
  * time       (time) datetime64[ns] 2008-01-01 ... 2012-12-31T23:00:00
Data variables:
    r          (time) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-02-18 17:48:23 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [11]:
ds_Era5 = xr.merge([df_RH_600, df_RH_650, df_RH_700, df_RH_750]) #start 17:50

In [12]:
ds_Era5

<xarray.Dataset>
Dimensions:    (time: 43848)
Coordinates:
    longitude  float32 204.5
    latitude   float32 19.75
  * time       (time) datetime64[ns] 2008-01-01 ... 2012-12-31T23:00:00
Data variables:
    r_600      (time) float32 ...
    r_650      (time) float32 ...
    r_700      (time) float32 ...
    r_750      (time) float32 ...

In [13]:
ds_merger = xr.merge([ds_Era5, CFHT_xr])

KeyboardInterrupt: 

In [ ]:
ds_Era5_monthly = ds_Era5.groupby('time.month').mean(dim='time')
ds_Era5_hourly = ds_Era5.groupby('time.hour').mean(dim='time')